In [3]:
import os
import sys
import psycopg2
from sqlalchemy import create_engine
from io import StringIO
import pandas as pd
import csv
up_one =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(up_one)
up_two =  (os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
sys.path.append(up_two)
#from minorlab import metapdb
import subprocess
from collections import defaultdict
from tqdm.notebook import trange, tqdm
import re
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)
pp1 = pprint.PrettyPrinter(indent=4, depth=1)

### Appris files and function

In [4]:
appris = pd.read_csv('Data/appris_transcript_names.tsv', sep='\t', header=None, names=['gene','ref'])
nonappris = pd.read_csv('Data/not_appris_transcript_names.tsv', sep='\t', header=None, names=['gene','notref'])
def ref_nonref(gene):
    ref = appris.loc [ appris['gene']== gene]['ref'].tolist()
    #print (ref)
    non_ref = nonappris.loc [nonappris['gene']==gene]['notref'].tolist()
    non_ref.sort()
    #print (non_ref)
    return (ref, non_ref)

In [7]:
appris

,gene,ref
0,DDX11L2,DDX11L2-202
1,DDX11L2,DDX11L2-201
2,DDX11L1,DDX11L1-201
3,WASH7P,WASH7P-201
4,MIR6859-1,MIR6859-1-201
...,...,...
80225,MT-ND6,MT-ND6-201
80226,MT-TE,MT-TE-201
80227,MT-CYB,MT-CYB-201
80228,MT-TT,MT-TT-201


In [7]:
print (ref_nonref('CD44'))

(['CD44-206'], ['CD44-201', 'CD44-202', 'CD44-203', 'CD44-204', 'CD44-205', 'CD44-207', 'CD44-208', 'CD44-209', 'CD44-210', 'CD44-211', 'CD44-212', 'CD44-213', 'CD44-214', 'CD44-215', 'CD44-216', 'CD44-217', 'CD44-218', 'CD44-219', 'CD44-220', 'CD44-221', 'CD44-222', 'CD44-223', 'CD44-224', 'CD44-225', 'CD44-226', 'CD44-227', 'CD44-228', 'CD44-229', 'CD44-230', 'CD44-231', 'CD44-232', 'CD44-233', 'CD44-234', 'CD44-235', 'CD44-236', 'CD44-237', 'CD44-238'])


In [ ]:
# Genes related to SMC differentiation
with open ('Data/temp_list.txt') as f:
    ff = set(f.read().splitlines())
print (ff)


# Read all Blast into DataFrame

In [8]:
## Read in full blast results  - Using version 2
# Note version 1 of blast_full_PDB_vs_gencode had some lines that were Warnings
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
blast_df= pd.read_csv('Data/blast_full_PDB_vs_gencode_v2.tsv', sep='\t', header = 1, names = columns)
# Reduce the sseqid to just isoform name
gene_hash={}
for iso in blast_df['sseqid']:
    p = iso.split('|')
    gene_hash[p[0]]= p[1].split('.')[0]
blast_df['sseqid'] = [ s.split('|')[0] for s in blast_df['sseqid'].to_list()]
blast_df['gene'] = [s.split('-')[0] for s in blast_df['sseqid'].to_list()]
blast_df

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident,gene
1,7evw_R,599,36,574,FZD1-201,647,103,647,0.000000e+00,905.0,551,440,18,6,93,79.855,FZD1
2,7evw_R,599,38,574,FZD2-201,565,28,565,0.000000e+00,893.0,553,434,31,3,88,78.481,FZD2
3,7evw_R,599,49,563,FZD5-201,585,33,536,2.420000e-180,523.0,533,276,47,14,210,51.782,FZD5
4,7evw_R,599,44,589,FZD10-201,581,29,573,1.480000e-165,485.0,562,261,33,10,268,46.441,FZD10
5,7evw_R,599,49,562,FZD3-201,666,28,512,2.230000e-162,480.0,515,236,31,3,248,45.825,FZD3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1909511,"2z9a_A,B",88,3,81,PSAP-201,524,311,389,7.630000e-47,157.0,79,79,0,0,0,100.000,PSAP
1909512,"2z9a_A,B",88,3,55,PSAP-204,167,115,167,4.070000e-31,108.0,53,53,0,0,0,100.000,PSAP
1909513,8d58_A,246,1,246,METTL1-202,276,20,265,0.000000e+00,516.0,246,246,0,0,0,100.000,METTL1
1909514,8d58_A,246,104,240,METTL1-204,124,2,124,3.650000e-85,250.0,137,123,14,1,0,89.781,METTL1


In [ ]:
blast_df.loc [blast_df['gene']=='BAZ2B'].sort_values(['qseqid','bitscore'], ascending = False)

# Read in the results from  max_ref_nonref_bitscores

In [9]:
# read in the results from  max_ref_nonref_bitscores
mrnr_df = pd.read_csv('Data/max_ref_nonref_bitscores.tsv', sep = '\t')
mrnr_df = mrnr_df.rename(columns = {'Unnamed: 0':'pdb'})
pdb_nonref_list_by_gene = mrnr_df.loc[ mrnr_df['diff'] > 10  ].groupby('gene')['pdb'].apply(list).to_dict()
pdb_ref_list_by_gene  = mrnr_df.loc[ mrnr_df['diff'] < 10 ].groupby('gene')['pdb'].apply(list).to_dict()
mrnr_df

,pdb,top,gene,top_nonref,max_nonref_bs,max_ref_bs,diff
0,6lth_L,4586.0,ARID1A,True,4586.0,3852.0,734.0
1,7ktr_C,1658.0,SUPT20H,True,1658.0,1163.0,495.0
2,"4kkd_A,B",892.0,MASP1,True,892.0,433.0,459.0
3,"7k36_B,D,E,F,G",1479.0,STRN3,True,1479.0,1046.0,433.0
4,8fgw_B,2585.0,IFT122,True,2585.0,2155.0,430.0
...,...,...,...,...,...,...,...
35076,"7t3p_A,B,C,D",5445.0,ITPR3,False,0.0,5445.0,-5445.0
35077,"7u9x_A,B,C,D",10359.0,RYR2,False,4861.0,10359.0,-5498.0
35078,"7u9q_A,D,B,C",10362.0,RYR2,False,4861.0,10362.0,-5501.0
35079,"6dqj_A,B,C,D",5566.0,ITPR3,False,0.0,5566.0,-5566.0


## Find genes with both ref and isos¶

In [10]:
both = set(pdb_nonref_list_by_gene).intersection(set(pdb_ref_list_by_gene))
#print (both)
results = []
for gene in list(both):
    #print (gene, ref_nonref(gene))
    mrnr_df2 = mrnr_df.loc[ mrnr_df['gene']== gene, ['pdb', 'gene', 'max_nonref_bs', 'max_ref_bs', 'diff']]
    max = mrnr_df2['diff'].max()
    min = mrnr_df2['diff'].min()
    #print (f'{gene}\t{max}\t{min}\t{max - min}' )
    results.append([gene, max, min, max-min])   
    #input ('ready?')   
#print (results)
mrnr_df3= pd.DataFrame(results, columns = ['gene', 'max', 'min','diff'])
check_first= mrnr_df3.loc [mrnr_df3['max'] > 50].sort_values(by='diff',ascending = False).reset_index(drop=True)
#check_first.to_excel('Data/check_first.xlsx')
check_first

,gene,max,min,diff
0,MASP1,459.0,-412.0,871.0
1,ARID1A,734.0,1.0,733.0
2,KCNN4,79.7,-630.0,709.7
3,SUPT20H,495.0,-158.0,653.0
4,AFF4,82.0,-563.0,645.0
...,...,...,...,...
93,CNGA3,54.0,0.0,54.0
94,DYSF,53.0,0.0,53.0
95,MED25,55.0,4.0,51.0
96,NRG1,50.2,0.0,50.2


In [ ]:
df = check_first.loc[ check_first['gene'] ==  'BAZ2B']
df

In [ ]:
check_2nd = mrnr_df3.loc [mrnr_df3['diff'] > 0].sort_values(by='diff',ascending = False).reset_index(drop=True)
check_2nd

## Read in Chess file

In [11]:
chess_pdb_dir = "C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2"
chess_df = pd.read_excel('Data/Chess/iso_all_v1.2.xlsx')
chess_df = chess_df.replace('-',np.nan)
chess_df['enst'] = [ str(g_id).split('.')[0] for g_id in chess_df['GENCODE ID']]
chess_dict = chess_df.set_index('enst')['CHESS ID'].dropna().to_dict()
chess_dict

{'ENST00000621850': 'CHS.10000.1',
 'ENST00000401611': 'CHS.10000.10',
 'ENST00000621128': 'CHS.10000.2',
 'ENST00000619839': 'CHS.10000.3',
 'nan': 'hypothetical.10127.1',
 'ENST00000316851': 'CHS.10000.alt102',
 'ENST00000531044': 'CHS.10000.5',
 'ENST00000611284': 'CHS.10000.51',
 'ENST00000529356': 'CHS.10000.alt103',
 'ENST00000526322': 'CHS.10000.alt106',
 'ENST00000533073': 'CHS.10000.alt107',
 'ENST00000524916': 'CHS.10000.alt108',
 'ENST00000530090': 'CHS.10000.alt109',
 'ENST00000528158': 'CHS.10000.alt110',
 'ENST00000529221': 'CHS.10003.alt104',
 'ENST00000494714': 'CHS.10003.alt105',
 'ENST00000531164': 'CHS.10003.alt103',
 'ENST00000464224': 'CHS.10003.alt106',
 'ENST00000314756': 'CHS.10003.alt110',
 'ENST00000527781': 'CHS.10003.alt111',
 'ENST00000525989': 'CHS.10003.alt112',
 'ENST00000534793': 'CHS.10003.alt113',
 'ENST00000478125': 'CHS.10003.alt114',
 'ENST00000480233': 'CHS.10003.alt115',
 'ENST00000303941': 'CHS.10004.alt100',
 'ENST00000542948': 'CHS.10004.alt10

In [ ]:

chess_dict['ENST00000392783']

## Next few cells adapted from Gloria


##### Read in the files

In [12]:
# Read in the appris transcripts
appris_transcripts = set()
with open('Data/appris_transcript_names.tsv', 'r') as appris_file:
    appris_reader = csv.reader(appris_file, delimiter='\t')
    for row in appris_reader:
        appris_transcripts.add(row[1])
    print (f'{len(appris_transcripts)} Appris transcripts')

# Read in the nonappris transcripts
nonappris_transcripts = set()
with open('Data/not_appris_transcript_names.tsv', 'r') as nonappris_file:
    nonappris_reader = csv.reader(nonappris_file, delimiter='\t')
    for row in nonappris_reader:
        nonappris_transcripts.add(row[1])
    print (f'{len(nonappris_transcripts)} non-Appris transcripts')

# Read in the PDB to GenCode mapping
pdb_to_gencode = []
with open('Data/blast_full_PDB_vs_gencode_v2.tsv', 'r') as pdb_file:
    pdb_reader = csv.reader(pdb_file, delimiter='\t')
    next(pdb_reader, None)
    for row in pdb_reader:
        # This makes the sseqid just the isoform name
        row[5] = row[5].split('|')[0]
        pdb_to_gencode.append(row)
    print (f'{len (pdb_to_gencode)} records in Gencode')

    


80169 Appris transcripts
141431 non-Appris transcripts
1909443 records in Gencode


### Group the PDB to GenCode mappings by qseqid

In [13]:
# Group the PDB to GenCode mappings by qseqid
# print (pdb_to_gencode[0])
pdb_grouped_by_qseqid = {}
for row in pdb_to_gencode:
    #print (row)
    qseqid = row[1]
    if qseqid not in pdb_grouped_by_qseqid:
        pdb_grouped_by_qseqid[qseqid] = []
    pdb_grouped_by_qseqid[qseqid].append(row)
print(f' This represents {len(pdb_grouped_by_qseqid)} pdbs')
# Below is just a check.
#for k in list(pdb_grouped_by_qseqid.keys())[0:2]:
#   print (pdb_grouped_by_qseqid[k])

 This represents 35081 pdbs


In [ ]:
pdb_grouped_by_qseqid['5wrv_A']

In [14]:
name_dict= {}
name_enst = {}
with open ('Data/transcript_list.txt') as file:
    transcript_names = file.read().splitlines()
for row in transcript_names:
    #print (row)
    parts=row.split('|')
    name_dict[parts[5]] = row
    name_enst[parts[5]] = parts[1]
#print (name_dict)
#print (name_enst)


### Filter the PDB to non-appris transcripts


In [15]:
# Filter the PDB to non-appris transcripts
filtered_pdb_to_gencode = []
count = 0
genes_with_ref=set()
also_refs = defaultdict(set)
ref_pdbs = defaultdict(list)
for qseqid, pdb_group in pdb_grouped_by_qseqid.items():
    #if pdb_group[0][5][0:5] != 'SRP68':
    #    continue
    #print (f'qseqid, pdb_group = {qseqid}, {pdb_group}')
    
    if pdb_group[0][5] in appris_transcripts:
        continue
    # Get the top hit based on bitscore
    # print (pdb_group)
    #top_hit = max(pdb_group, key=lambda row:float(row[10]))
    #print (f'top_hit = {top_hit}')
    top_hit=[]
    top_bitscore = 0.
    for pg in pdb_group:
        if float(pg[10]) > top_bitscore:
            top_hit = pg
            
    top_bitscore = float(top_hit[10])
    bitscore_threshold = top_bitscore * 0.9
    
    # Get all other hits with the same bitscore as the top hit
    ## Note other_hits will now include all that match top_bitscore 
    hits = [row for row in pdb_group if float(row[10]) == top_bitscore]
    second_tier = [row for row in pdb_group if bitscore_threshold < float(row[10]) < top_bitscore ]
    
    # Determine if all sseqids are nonappris-
    all_nonappris = all(row[5] in nonappris_transcripts for row in hits)
    
    # If all sseqids are nonappris, add the gene name to each row and output to the filtered list
    if all_nonappris:
        gene_name = top_hit[5].split('-')[0]
        #top_hit_with_gene = top_hit + [gene_name]
        #filtered_pdb_to_gencode.append(top_hit_with_gene)
        for hit in hits:
            hit_with_gene = hit + [gene_name]
            filtered_pdb_to_gencode.append(hit_with_gene)
        for hit in second_tier:
            if hit[5] in appris_transcripts:
                genes_with_ref.add(gene_name)
                also_refs[gene_name].add(hit[5] )
                ref_pdbs[gene_name].append(hit[1] )
    count += 1
    if count == 3:
        pass#break
print (f'filtered_pdb_to_gencode is {len(filtered_pdb_to_gencode)} long')

#print (filtered_pdb_to_gencode)

filtered_pdb_to_gencode is 8230 long


In [12]:
filtered_pdb_to_gencode[0:10]

[['517',
  '7e8h_I,K',
  '760',
  '24',
  '119',
  'DPP10-206',
  '97',
  '2',
  '96',
  '1.03e-22',
  '94.7',
  '96',
  '43',
  '1',
  '1',
  '52',
  '44.792',
  'DPP10'],
 ['842',
  '2fy2_A',
  '612',
  '471',
  '605',
  'CRAT-217',
  '551',
  '422',
  '547',
  '6.30e-24',
  '107',
  '135',
  '56',
  '9',
  '2',
  '70',
  '41.481',
  'CHAT'],
 ['866',
  '2fy2_A',
  '612',
  '262',
  '309',
  'CHAT-210',
  '72',
  '1',
  '48',
  '7.71e-28',
  '107',
  '48',
  '48',
  '0',
  '0',
  '0',
  '100.000',
  'CHAT'],
 ['949',
  '6b0b_A,E',
  '186',
  '4',
  '183',
  'APOBEC3B-203',
  '490',
  '9',
  '188',
  '3.47e-21',
  '91.3',
  '187',
  '59',
  '14',
  '4',
  '114',
  '31.551',
  'APOBEC3B'],
 ['999',
  '6vwb_A',
  '90',
  '37',
  '90',
  'MUS81-202',
  '389',
  '2',
  '55',
  '1.60e-30',
  '112',
  '54',
  '54',
  '0',
  '0',
  '0',
  '100.000',
  'MUS81'],
 ['1109',
  '4kmn_A',
  '103',
  '4',
  '43',
  'BIRC2-205',
  '262',
  '220',
  '259',
  '4.88e-21',
  '85.5',
  '40',
  '40',
  '0

##### Get some statistics

In [17]:
#print(filtered_pdb_to_gencode[0:3])
print (f'filtered_pdb_to_gencode is {len(filtered_pdb_to_gencode)} records long')
pdb_set= set()
gene_set= set()
for group in filtered_pdb_to_gencode:
    pdb_set.add(group[1])
    gene_set.add(group[17])
num_pdb_refs = sum ([len (also_refs[gene]) for gene in also_refs])
print (f'The reduced set of pdb_matches comprises {len(pdb_set)} PDBs that match {len (gene_set)} genes.')
print (f'For these genes, {len(genes_with_ref)} genes also have well-matched (bitscore >90% top bitscore) reference isoform ({num_pdb_refs} isoforms).'  )

filtered_pdb_to_gencode is 8230 records long
The reduced set of pdb_matches comprises 6249 PDBs that match 2077 genes.
For these genes, 109 genes also have well-matched (bitscore >90% top bitscore) reference isoform (225 isoforms).


In [ ]:
just_alt = gene_set - genes_with_ref
print (len(just_alt) )
print (f'These have just_alt\n{just_alt}')


In [20]:
combined_filtered =  [sublist for sublist in filtered_pdb_to_gencode ]
#print (combined_filtered)
columns =  ['index','qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident', 'gene']
non_appris_pdbs =pd.DataFrame(combined_filtered, columns=columns)
non_appris_pdbs =non_appris_pdbs.drop(columns=['index'])
non_appris_pdbs = non_appris_pdbs.sort_values(by=['gene','sseqid','qseqid'])
non_appris_pdbs

,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident,gene
3745,5t5s_A,215,7,215,AARS1-203,999,757,996,7.13e-135,404,240,208,31,1,1,86.667,AARS1
5384,4xem_A,475,21,68,AARS1-235,48,1,48,1.30e-26,102,48,48,0,0,0,100.000,AARS1
5383,"4xeo_A,B",475,21,68,AARS1-235,48,1,48,1.40e-26,102,48,48,0,0,0,100.000,AARS1
7937,"5knn_A,B,C,D,E,F,G,H",450,1,45,AARS1-235,48,4,48,7.22e-25,97.8,45,45,0,0,0,100.000,AARS1
1741,6aam_A,298,6,211,AATK-202,879,118,331,4.60e-21,94.7,220,70,20,6,130,31.818,AATK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919,5kl6_A,93,6,91,ZNF544-215,573,459,540,8.45e-21,86.3,86,42,4,2,40,48.837,ZNF544
3876,6vtx_A,84,8,83,ZNF544-215,573,439,512,7.86e-21,85.9,76,38,2,1,36,50.000,ZNF544
1918,4r2e_A,93,5,91,ZNF565-207,259,154,236,9.78e-21,84.0,87,42,4,2,41,48.276,ZNF565
2071,7z36_D,70,1,68,ZNF607-203,102,4,71,8.18e-21,79.3,68,39,0,0,29,57.353,ZNF607


### Function to extract sequences of Isoforms and related sequences
USE: get_gene_seqs (gene, df=found_df)  
By default, will extract data from found_df, but can be specified  
##### default_df is currently non_appris_pdbs 


In [ ]:
default_df

In [18]:
df4 = pd.read_csv('Data/top2_pdb_vs_gencode.tsv', sep='\t')
df4 = df4.drop(columns =['Unnamed: 0'])
#df4['sseqid'] = [s.split('|')[0] for s in df4['sseqid'].to_list()]
my_list = []
for s in df4['sseqid'].to_list():
    my_list.append (str(s).split('|')[0])
print (my_list[0:10])
df4['sseqid']= my_list
df4['gene']= [s.split('-')[0] for s in df4['sseqid'].to_list()]
df4

['FZD7-201', 'FZD1-201', 'PTGR2-201', 'PTGR2-204', 'FAU-203', 'FAU-205', 'MAPK1-202', 'MAPK1-201', 'DPP6-203', 'DPP6-201']


,qseqid,qlen,qstart,qend,sseqid,slen,sstart,send,evalue,bitscore,length,nident,gaps,gapopen,mismatch,pident,gene
0,7evw_R,599.0,38.0,574.0,FZD7-201,574.0,38.0,574.0,0.000000e+00,1115.0,537.0,537.0,0.0,0.0,0.0,100.000,FZD7
1,7evw_R,599.0,36.0,574.0,FZD1-201,647.0,103.0,647.0,0.000000e+00,905.0,551.0,440.0,18.0,6.0,93.0,79.855,FZD1
2,2zb4_A,357.0,7.0,357.0,PTGR2-201,351.0,1.0,351.0,0.000000e+00,728.0,351.0,351.0,0.0,0.0,0.0,100.000,PTGR2
3,2zb4_A,357.0,7.0,357.0,PTGR2-204,351.0,1.0,351.0,0.000000e+00,728.0,351.0,351.0,0.0,0.0,0.0,100.000,PTGR2
4,7tql_e,51.0,4.0,51.0,FAU-203,98.0,44.0,98.0,1.560000e-23,85.5,55.0,46.0,7.0,1.0,2.0,83.636,FAU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69093,4z0x_A,105.0,1.0,93.0,IGLV3-21-201,117.0,21.0,113.0,1.250000e-50,156.0,93.0,74.0,0.0,0.0,19.0,79.570,IGLV3
69094,"2z9a_A,B",88.0,3.0,81.0,PSAP-201,524.0,311.0,389.0,7.630000e-47,157.0,79.0,79.0,0.0,0.0,0.0,100.000,PSAP
69095,"2z9a_A,B",88.0,3.0,55.0,PSAP-204,167.0,115.0,167.0,4.070000e-31,108.0,53.0,53.0,0.0,0.0,0.0,100.000,PSAP
69096,8d58_A,246.0,1.0,246.0,METTL1-202,276.0,20.0,265.0,0.000000e+00,516.0,246.0,246.0,0.0,0.0,0.0,100.000,METTL1


In [19]:
print (ref_nonref('CD44'))

(['CD44-206'], ['CD44-201', 'CD44-202', 'CD44-203', 'CD44-204', 'CD44-205', 'CD44-207', 'CD44-208', 'CD44-209', 'CD44-210', 'CD44-211', 'CD44-212', 'CD44-213', 'CD44-214', 'CD44-215', 'CD44-216', 'CD44-217', 'CD44-218', 'CD44-219', 'CD44-220', 'CD44-221', 'CD44-222', 'CD44-223', 'CD44-224', 'CD44-225', 'CD44-226', 'CD44-227', 'CD44-228', 'CD44-229', 'CD44-230', 'CD44-231', 'CD44-232', 'CD44-233', 'CD44-234', 'CD44-235', 'CD44-236', 'CD44-237', 'CD44-238'])


In [ ]:
[s.split('|')[0] for s in df4['sseqid']]

In [26]:
pdb_file='Data/pdb_unique_human_seqs_230301.fa'
gene_file='Data/gencode_v43_translations_mod.fa'
default_df = df4
def extract_sequence(filename, sequence_id):
    sequence = ""
    with open(filename, "r") as f:
        for line in f:
            if line.startswith(">") and sequence_id in line:
                if '|' in line:
                    sequence += line.split('|')[0] + '\n'
                else:
                     sequence += line
                # Found the sequence ID we want
                while True:
                    line = f.readline().strip()
                    if line.startswith(">") or not line:
                        # Reached the end of the sequence
                        break
                    sequence += line
    sequence += "\n"
    return sequence
def get_gene_seqs (gene, df=default_df, field='sseqid'):
    matches = df.loc[ df[field].str.contains(gene) ]
    isos = set()
    pdbs = set()
    for i,row in matches.iterrows():
        isos.add(row['sseqid'])
        pdbs.add(row['qseqid'])
    seqs=""
    for pdb in pdbs:
        seqs+= extract_sequence (pdb_file, pdb)
    for iso in isos:
        seqs+= extract_sequence (gene_file, iso)
    return (seqs)
def get_all_gene_seq(gene):
    ref,nonref = ref_nonref(gene)
def pdbs_with_appris (gene, df=default_df):
    matches = df.loc[ df['sseqid'].str.contains(gene) ]
    pdbs = set()
    genes = set()
    isos = set()
    for i,row in matches.iterrows():
        pdbs.add(row['qseqid'])
        genes.add(row['gene'])
        isos.add(row['sseqid'])
    #print (pdbs, genes)
    seqs=""
    for pdb in pdbs:
        seqs+= extract_sequence (pdb_file, pdb)
    for gene in genes:
        to_get, dont_get = ref_nonref(gene)
        if to_get:
            seqs += extract_sequence (gene_file, to_get[0]).replace('>','>REF_')
    for iso in isos:
        seqs+= extract_sequence (gene_file, iso).replace('>','>ALT_')
        
        #if gene in genes_with_ref:
        #    for iso in also_refs[gene]:
        #        seqs += extract_sequence(gene_file, iso).replace('>','>REF_')
        #    for pdb in ref_pdbs[gene]:
        #        seqs += extract_sequence(pdb_file, pdb).replace('>','>REF_')
    return (seqs)



s=pdbs_with_appris('TARBP2')
print (s.splitlines()[0:3])

['>4wyq_B,E', 'ALGPACCRVLSELSEEQAFHVSYLDIEELSLSGLCQCLVELSTQPATVCHGSATTREAARGEAARRALQYLKIMA', '>6zbk_B']


In [27]:
def get_all_gene_seqs(gene):
    ref,nonref = ref_nonref(gene)
    to_return = ""
    for r in ref:
        r_seq =  extract_sequence(gene_file,r).replace('>',">ref_")
        to_return += r_seq
    for n in nonref:
        n_seq = extract_sequence(gene_file,n).replace('>',">alt_")
        to_return += n_seq
    return to_return
print (get_all_gene_seqs('CD44'))

>ref_CD44-206
MDKFWWHAAWGLCLVPLSLAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDVSSGSSSERSSTSGGYIFYTFSTVHPIPDEDSPWITDSTDRIPATSTSSNTISAGWEPNEENEDERDRHLSFSGSGIDDDEDFISSTISTTPRAFDHTKQNQDWTQWNPSHSNPEVLLQTTTRMTDVDRNGTTAYEGNWNPEAHPPLIHHEHHEEEETPHSTSTIQATPSSTTEETATQKEQWFGNRWHEGYRQTPKEDSHSTTGTAAASAHTSHPMQGRTTPSPEDSSWTDFFNPISHPMGRGHQAGRRMDMDSSHSITLQPTANPNTGLVEDLDRTGPLSMTTQQSNSQSFSTSHEGLEEDKDHPTTSTLTSSNRNDVTGGRRDPNHSEGSTTLLEGYTSHYPHTKESRTFIPVTSAKTGSFGVTAVTVGDSNSNVNRSLSGDQDTFHPSGGSHTTHGSESDGHSHGSQEGGANTTSGPIRTPQIPEWLIILASLLALALILAVCIAVNSRRRCGQKKKLVINSGNGAVEDRKPSGLNGEASKSQEMVHLVNKESSETPDQFMTADETRNLQNVDMKIGV
>alt_CD44-201
MDKFWWHAAWGLCLVPLSLAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDVSSGSSSERSSTSGGYIFYTFSTVHPIPDEDSPWITDSTDRIPATRDQDTFHPSGGSHTTHGSESDGHSHGSQEGGANTTSGPIRTPQIPEWLII

In [ ]:
#mrnr_df
pdbs_with_appris('DCX',df4)

### Publications


In [23]:
publications = pd.read_csv('Data/pdb_publications.tsv', sep = '\t' )
publications

,pdbid,pdb_id,title,year,pmid,doi,journal_abbrev,volume,pages,journal_issn
0,1wlp,1WLP,NMR solution structure of the tandem Src homol...,2006.0,16326715.0,10.1074/jbc.M505193200,J.Biol.Chem.,281,3660-3668,0021-9258
1,1wo4,1WO4,Assessment of the robustness of a serendipitou...,2005.0,15698569.0,10.1016/j.str.2004.12.007,Structure,13,257-266,0969-2126
2,1wso,1WSO,Orexin-A is composed of a highly conserved C-t...,2006.0,16429482.0,10.1002/psc.747,J.Pept.Sci.,12,443-454,1075-2617
3,1wtg,1WTG,Novel interactions of large P3 moiety and smal...,2005.0,15607748.0,10.1016/j.bbrc.2004.11.108,Biochem.Biophys.Res.Commun.,326,859-865,0006-291X
4,1wvo,1WVO,Solution structure of the antifreeze-like doma...,2006.0,16597820.0,10.1110/ps.051700406,Protein Sci.,15,1010-1016,0961-8368
...,...,...,...,...,...,...,...,...,...,...
52982,1wug,1WUG,Selective small molecules blocking HIV-1 Tat a...,2005.0,15724976.0,10.1021/ja044885g,J.Am.Chem.Soc.,127,2376-2377,0002-7863
52983,1wum,1WUM,Selective small molecules blocking HIV-1 Tat a...,2005.0,15724976.0,10.1021/ja044885g,J.Am.Chem.Soc.,127,2376-2377,0002-7863
52984,1wuu,1WUU,Molecular Structure of Human Galactokinase: IM...,2005.0,15590630.0,10.1074/jbc.M412916200,J.Biol.Chem.,280,9662-9670,0021-9258
52985,1wou,1WOU,Structural basis of cellular redox regulation ...,2004.0,15355959.0,10.1074/jbc.M407079200,J.Biol.Chem.,279,48120-48125,0021-9258


In [24]:
if '4lsd' in publications['pdbid']:
    print ('published')

In [25]:
#for just_alt, get pdbs and return t/f if in publications
pdb_list = []
pubs = []
unpublished = set()
for gene in list(just_alt):
    published = 0 
    #print (gene)
    pdbs_for_genes_df = non_appris_pdbs[ non_appris_pdbs['sseqid'].str.startswith(gene)]
    #print (f'{len(pdbs_for_genes_df)} pdbs')
    for pdb in pdbs_for_genes_df['qseqid']:
        #print (pdb, end ="")
        if pdb[0:4] in publications['pdbid'].to_list():
            published = 1
            #print (f' published')
        #else:
            #print (' unpublished')
    #print()
    #print (f'pdb_list for {gene} is {pdb_list}')
    if published == 0:
        unpublished.add(gene)
#pdbs_for_genes_df
#pdbs_for_genes_df = non_appris_pdbs[ non_appris_pdbs['sseqid'].str.startswith('DCD')]
print (f'unpublished list is {unpublished}')
#print (pdb_list)

NameError: name 'just_alt' is not defined

## MSA 

In [28]:
#data_dir = 'Data/Non_appris_seqs'
data_dir= 'Data/Seq_groups'
#data_dir='Data/Aln_files'
def one_msa(g):
    fasta_file = f'{data_dir}/{g}.fa'
    if not os.path.isfile(fasta_file):
        s=get_gene_seqs(g)
        with open (fasta_file, 'w') as out:
            out.write(s)
    msa_file = f'{data_dir}/{g}.aln'
    if not os.path.isfile(msa_file):
        cmd = f'clustalo -i {fasta_file} --outfmt=clu -o {msa_file}'
        print (cmd)
        r=subprocess.getoutput(cmd)
        if r[0:5] =="ERROR":
            return (f'Something went wrong with {g}')
        print (r)
    with open (msa_file, 'r') as f:
        rr=f.read()
        print (rr)    
    
def show_msa(gene_list, fresh=0, last=None):
    '''Will generate and show a MSA for a list of genes. If last is set, it will skip until it reaches that gene'''
    if len(gene_list)==1:
        fresh =1
    for g in gene_list:
        if fresh==0 and g!=last:
            continue
        print (g)
        print (f'Reference isoform is {ref_nonref(g)[0]}')
        fresh=1
        last = g
        fasta_file = f'{data_dir}/{g}.fa'
        if not os.path.isfile(fasta_file):
            s=get_gene_seqs(g)
            with open (fasta_file, 'w') as out:
                out.write(s)
        msa_file = f'{data_dir}/{g}.aln'
        if not os.path.isfile(msa_file):
            cmd = f'clustalo -i {fasta_file} --outfmt=clu -o {msa_file}'
            print (cmd)
            r=subprocess.getoutput(cmd)
            if r[0:5] =="ERROR":
                print (f'Something went wrong with {g}')
                continue
            print (r)
        with open (msa_file, 'r') as f:
            rr=f.read()
            print (rr)

        #read = input ("Ready to move on? f to see sequence file ")
        #if read.lower() == 'f':
            with open (fastafile) as seqs:
                print (seqs.read())
            if len (gene_list) > 1:
                read = input ("Ready to move on? ")
            
def show_seqs_loop(gene_list, fresh=0, last=None):
    #expects a df with the name of the gene in position 0
    for g in gene_list:
        #print (g)
        if fresh==0 and g!=last:
            continue
        fresh=1
        last = g   
        with open (f'Data/Seq_groups/{g}.fa') as seqs:
            print(seqs.read())
        read = input ("Ready to move on? f to see sequence file")

def show_seqs(gene):
    with open (f'Data/Seq_groups/{gene}.fa') as seqs:
        print (seqs.read())

def matches_for_gene(gene, df = default_df):
    df2 = df.loc[df['sseqid'].str.startswith(gene)]
    return (df2)

In [29]:
show_seqs('CD44')

>1poz_A
AQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDV
>4pz3_A,B
AMAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYP
>4pz4_A,B
AMAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPS
>2i83_A
MAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDV
>CD44-211|ENST00000442151.6
XDKFWWHAAWGLCLVPLSLAQIDLNITCRFAGVFHVEKNGRYSISRTEAADLCKAFNSTLPTMAQMEKALSIGFETCRYGFIEGHVVIPRIHPNSICAANNTGVYILTSNTSQYDTYCFNASAPPEEDCTSVTDLPNAFDGPITITIVNRDGTRYVQKGEYRTNPEDIYPSNPTDDDVSSGSSSERSSTSGGYIFYTFSTVHPIPDEDSPWITDSTDRIPATRDQDTFHPSGGSHTTHGSESDGHSHGSQEGGANTTSGPIRTPQIPEWLIILASLLALALILAVCIAVNSRRS
>CD44-214|E

In [ ]:
matches_for_gene('IL1RAP')  #
#matches_for_gene('AFF4', df=blast_df)

### Create a directory with sequence files for all the matches

In [ ]:
failed = []
dir = 'Data/Non_appris_seqs'
for gene in set (non_appris_pdbs['gene'].tolist()):
    file = f'{dir}/{gene}.fa'
    if os.path.isfile(file):
        continue
    try:
        s = pdbs_with_appris(gene)
    except:
        print (f'Something went wrong with {gene}')
        failed.append(gene)
    with open (file, 'w') as out:
        out.write(s)
if failed:
    print ("!!!!   !!!! Some things failed. Writen to failed_files.txt")
    with open (f'{dir}/failed_files.txt', 'w') as failed:
       failed.write(failed)
           
print ('Done')

In [30]:
just_nonref = set (pdb_nonref_list_by_gene)
has_both = set (pdb_nonref_list_by_gene).intersection(set(pdb_ref_list_by_gene))
print (has_both)
    

{'TTN', 'CUTA', 'AFF4', 'SNAP25', 'SF3B4', 'ARID1A', 'ATP6V1H', 'VEGFA', 'PDE5A', 'NRP2', 'SLC12A6', 'SMARCA2', 'PDE4D', 'DMPK', 'KAT6A', 'ECT2', 'KCNN4', 'CR2', 'API5', 'RANBP3', 'NFYC', 'FGFR1', 'OTUD5', 'TFPI', 'PTK2', 'ANK1', 'ABCG1', 'PSMC3', 'CSF2RB', 'GOLGA2', 'PIK3R1', 'FGFR3', 'FXYD2', 'SPAST', 'HNRNPA2B1', 'ATP2A2', 'MAPK8', 'SEH1L', 'CERT1', 'MAT2B', 'ENAH', 'NTRK3', 'UBE2V1', 'AP1S3', 'ATP13A2', 'GABRG2', 'CAMK2G', 'CCR2', 'IGHM', 'NT5M', 'SULT2B1', 'AGAP3', 'BTRC', 'RICTOR', 'RAC1', 'BRD4', 'SIRPB1', 'BRD9', 'RAB11FIP3', 'LGALS9', 'RGS7', 'MGAM', 'NELFCD', 'TMPRSS13', 'DUSP13', 'ZFP64', 'NASP', 'CSF3', 'SLC12A5', 'GOLGA4', 'EZH2', 'PTEN', 'PXN', 'FN1', 'MCM3', 'MED16', 'CASP8', 'MACF1', 'GABRB3', 'MGLL', 'LRP8', 'PGM1', 'GIGYF2', 'ACLY', 'TRRAP', 'KDM1A', 'ADCYAP1R1', 'VPS29', 'GCK', 'CSF1', 'IMPDH1', 'PPFIA2', 'PAK6', 'NEDD4', 'GSN', 'ITGB1', 'DNM1', 'TRIO', 'PTCH1', 'DNMT3B', 'HNRNPK', 'VAV1', 'GRIA2', 'TUBGCP2', 'PSMA1', 'KCND3', 'DDR1', 'IL1RAP', 'PI4KB', 'INS', 'GNAS'

In [30]:
def get_af2s(gene):
    to_return = []
    ref,nonref = ref_nonref(gene)
    #df = blast_df.loc[ blast_df['qseqid'].isin(pdb_nonref_list_by_gene[gene])]
    df = blast_df.loc[ blast_df['qseqid'].isin(nonref)]
    df2 = df.loc[df.groupby('qseqid')['bitscore'].idxmax()]
    af2_ids = df2['sseqid'].map(gene_hash).unique()
    if len(af2_ids) > 0:
        for id in af2_ids:
            if id in chess_dict:
                file=f'{chess_pdb_dir}/{chess_dict[id]}.pdb'
                local_file = f'Data/Chess/pdb_v1.2/{chess_dict[id]}.pdb'
                if os.path.isfile(local_file):
                    to_return.append(file)
    if to_return:
        return to_return
    else:
        print ("NO AF2 Models found")
        return 0
        
print (get_af2s('CD44'))



NO AF2 Models found
0


In [31]:
def get_all_af2s(gene):
    to_return = []
    name_key = {}
    ensts = []
    ref,nonref = ref_nonref(gene)
    # Get ref[0]
    ensts.append( name_enst [ref[0]].split('.')[0] )
    #print (f"ref is {ensts[0].split('.')[0]}")
    for nr in nonref:
        if nr in name_enst:
            ensts.append( name_enst[nr].split('.')[0] )
            #print (f"nr is {name_enst[nr].split('.')[0]}") 
    for id in ensts:
        if id in chess_dict:
            file=f'{chess_pdb_dir}/{chess_dict[id]}.pdb'
            local_file = f'Data/Chess/pdb_v1.2/{chess_dict[id]}.pdb'
            if os.path.isfile(local_file):
                to_return.append(file)
    if to_return:
        return to_return
    else:
        print ("NO AF2 Models found")
        return 0
        
print (get_all_af2s('CD44'))
    

['C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.3.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.6.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.alt117.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.7.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.alt116.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.4.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.5.pdb', 'C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.84

## ChimeraX stuff

In [ ]:
os.getcwd()

In [32]:
# Relies on blast_df
chimerax_dir = 'C:/Users/daroc/Documents/Work/Structures/Chimerax'
posix_chimerax_dir = '/mnt/c/Users/daroc/Documents/Work/Structures/Chimerax'

color_pairs = [('rgb(35,45,75)', 'medium blue') , ('rgb(229,114,0)', 'orange' ), ('dark green','lime green'), ('yellow','khaki'),
               ('indigo','blue violet'), ('maroon', 'firebrick'), ('dark turquoise', 'cyan'), ('saddle brown','chocolate'),
               ('dodger blue', 'sky blue'), ('deep pink', 'hot pink'), ('dim grey', 'grey'), ('olive','chartreus')              
              ]
color_pairs = color_pairs + color_pairs

def split_chains(pdbs):
    to_return = []
    for pdb in pdbs: 
        #print (pdb)
        p= re.split('_|,',pdb)
        to_return.append((p[0],p[1]))
    return ( to_return)
def get_ref_pdb(gene):
    """Finds longest reference & returns a 1 ently list. The sole record is (pdb,chain)"""
    ref_blast = blast_df.loc[ blast_df['qseqid'].isin(pdb_ref_list_by_gene[gene])]
    longest_ref = ref_blast.loc[ref_blast['length'].idxmax()]['qseqid']
    print (f'The longest reference isoform is {longest_ref}')
    return split_chains([longest_ref])
def get_iso_pdbs(gene):
    """Returns a dict {pdb:chain} for isos with distinct length"""
    ref,nonref = ref_nonref(gene)
    #df = blast_df.loc[ blast_df['qseqid'].isin(pdb_nonref_list_by_gene[gene])]
    df = blast_df.loc[ blast_df['qseqid'].isin(nonref)]
    df_lowest_index_by_length = df.groupby('length')['length'].idxmin()
    #print (df_lowest_index_by_length)
    result_df = df.loc[df_lowest_index_by_length]
    isos =  split_chains(result_df['qseqid'].unique())
    return isos
def get_all_iso_pdbs(gene):
    """Returns a dict {pdb:chain} for isos with distinct length"""
    ref,nonref = ref_nonref(gene)
    df = blast_df.loc[ blast_df['qseqid'].isin(nonref)]
    isos =  split_chains(df['qseqid'].unique())
    return isos   
#print (get_all_iso_pdbs('CCR2'))
def get_af2s(gene):
    to_return = []
    ref,nonref = ref_nonref(gene)
    #df = blast_df.loc[ blast_df['qseqid'].isin(pdb_nonref_list_by_gene[gene])]
    df = blast_df.loc[ blast_df['qseqid'].isin(nonref)]
    df2 = df.loc[df.groupby('qseqid')['bitscore'].idxmax()]
    af2_ids = df2['sseqid'].map(gene_hash).unique()
    if len(af2_ids) > 0:
        for id in af2_ids:
            if id in chess_dict:
                file=f'{chess_pdb_dir}/{chess_dict[id]}.pdb'
                local_file = f'Data/Chess/pdb_v1.2/{chess_dict[id]}.pdb'
                if os.path.isfile(local_file):
                    to_return.append(file)
    if to_return:
        return to_return
    else:
        print ("NO AF2 Models found")
        return 0
    
def get_all_af2s(gene):
    to_return = []
    name_key = {}
    ensts = []
    ref,nonref = ref_nonref(gene)
    # Get ref[0]
    ensts.append( name_enst [ref[0]].split('.')[0] )
    #print (f"ref is {ensts[0].split('.')[0]}")
    for nr in nonref:
        if nr in name_enst:
            ensts.append( name_enst[nr].split('.')[0] )
            #print (f"nr is {name_enst[nr].split('.')[0]}") 
    for id in ensts:
        if id in chess_dict:
            file=f'{chess_pdb_dir}/{chess_dict[id]}.pdb'
            local_file = f'Data/Chess/pdb_v1.2/{chess_dict[id]}.pdb'
            if os.path.isfile(local_file):
                to_return.append(file)
    if to_return:
        return to_return
    else:
        print ("NO AF2 Models found")
        return 0      
    
        
def chimerax(gene):
    # Reference isoform stuff
    ref_pdb = get_ref_pdb(gene)

    #print (ref_pdb, isos)
    cx = f"""close session
cd C:/Users/daroc/Documents/Work/Structures/Chimerax
set bgcolor white
ui mousemode right pivot
"""
    
    cx += f"""open {ref_pdb[0][0]}
 hide #1 cartoons
 hide #1 atoms
 show #1/{ref_pdb[0][1]} cartoons
 color #1/{ref_pdb[0][1]} {color_pairs[0][0]}
"""
    # Iso forms
    isos = get_all_iso_pdbs(gene)
    count = 2
    for iso in isos:
        cx += f"""open {iso[0]}
 hide #{count} cartoons
 hide #{count} atoms
 show #{count}/{iso[1]} cartoons
 mm #{count}/{iso[1]} to #1/{ref_pdb[0][1]}
 color #{count}/{iso[1]} {color_pairs[count-1][0]}
"""
        count += 1


    # AF2 stuff
    af2s = get_all_af2s(gene)
    if af2s:
        for af2 in af2s:
            cx += f"""
open "{af2}"
 hide #{count} cartoons
 hide #{count} atoms
 show #{count} cartoons
 mm #{count} to #1/{ref_pdb[0][1]}
 color #{count} {color_pairs[count-1][0]}
        
"""
            count += 1
# END AF2 Stuff 

    cx += f"""open "c:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/{data_dir}/{gene}.aln"\n """
    cx += f"""view\n #Structres for {gene}
save "C:/Users/daroc/Documents/Work/Structures/Chimerax/{gene}.png"  transparentBackground  true
"""
    if os.name == 'posix':
        cxdir =posix_chimerax_dir
    else:
        cxdir = chimerax_dir
    cxc_file = f'{cxdir}/{gene}.cxc'
    with open (cxc_file, 'w') as out:
        out.write(cx)
    print (f"""ChimeraX script written. Use open "{chimerax_dir}/{gene}.cxc" """)    
    return (cx)
    
    

    
cxc = chimerax('CD44')
print (cxc)

The longest reference isoform is 2i83_A
ChimeraX script written. Use open "C:/Users/daroc/Documents/Work/Structures/Chimerax/CD44.cxc" 
close session
cd C:/Users/daroc/Documents/Work/Structures/Chimerax
set bgcolor white
ui mousemode right pivot
open 2i83
 hide #1 cartoons
 hide #1 atoms
 show #1/A cartoons
 color #1/A rgb(35,45,75)

open "C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.3.pdb"
 hide #2 cartoons
 hide #2 atoms
 show #2 cartoons
 mm #2 to #1/A
 color #2 rgb(229,114,0)
        

open "C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.6.pdb"
 hide #3 cartoons
 hide #3 atoms
 show #3 cartoons
 mm #3 to #1/A
 color #3 dark green
        

open "C:/Users/daroc/OneDrive - University of Virginia/Sheynkman/Python/Pdb-isoform-analysis/Data/Chess/pdb_v1.2/CHS.8449.alt117.pdb"
 hide #4 cartoons
 hide #4 atoms
 show #4 cartoons
 mm #4 to #1/A
 color #

In [33]:
def chess_for_gene(gene, print_it = True):
    to_return = {}
    ref,nonref = ref_nonref(gene)
    for name in ref:
        print (name)
        enst = name_enst[name].split('.')[0]
        print (f'ENST is {enst}')
        if enst in chess_dict:
            chess = chess_dict[enst]
            to_return[f'ref_{name}'] = chess
            print (f'Chess ref is {chess}')
    for name in nonref:
        if name in name_enst:
            enst = name_enst[name].split('.')[0]
            if enst in chess_dict:
                chess = chess_dict[enst]
                to_return[f'alt_{name}'] =  chess
                
    if print_it:
        for i,v in to_return.items():
            print (f'{i}\t{v}')
    return to_return
                      
name_key = chess_for_gene('CD44')


CD44-206
ENST is ENST00000415148
Chess ref is CHS.8449.3
ref_CD44-206	CHS.8449.3
alt_CD44-201	CHS.8449.6
alt_CD44-203	CHS.8449.alt117
alt_CD44-205	CHS.8449.7
alt_CD44-208	CHS.8449.alt116
alt_CD44-209	CHS.8449.4
alt_CD44-210	CHS.8449.5
alt_CD44-211	CHS.8449.1
alt_CD44-222	CHS.8449.alt100


In [79]:
print (name_enst['CD44-206'])
print (chess_dict['ENST00000415148'])
print (ref_nonref('CD44'))

ENST00000415148.6
CHS.8449.3
(['CD44-206'], ['CD44-201', 'CD44-202', 'CD44-203', 'CD44-204', 'CD44-205', 'CD44-207', 'CD44-208', 'CD44-209', 'CD44-210', 'CD44-211', 'CD44-212', 'CD44-213', 'CD44-214', 'CD44-215', 'CD44-216', 'CD44-217', 'CD44-218', 'CD44-219', 'CD44-220', 'CD44-221', 'CD44-222', 'CD44-223', 'CD44-224', 'CD44-225', 'CD44-226', 'CD44-227', 'CD44-228', 'CD44-229', 'CD44-230', 'CD44-231', 'CD44-232', 'CD44-233', 'CD44-234', 'CD44-235', 'CD44-236', 'CD44-237', 'CD44-238'])


## Multi-output cell

In [ ]:
{'CCDC92', 'PRDM16', 'ITGA1', 'MRAS', 'EHMT2', 'CENPW', 'NOB1', 'NT5C2', 'PARP12', 'ZNF664', 'SMG6', 'RP11-214K3.25', 'DHX58', 'RAB5C', 'SREBF1', 'OAZ3', 'WIPI1'}

In [ ]:
print (chimerax(gene))

In [82]:
gene = 'CD44'

print (f"""open "C:/Users/daroc/Documents/Work/Structures/Chimerax/{gene}.cxc" """)
print (ref_nonref(gene))

print ("Matchces for gene\n", matches_for_gene(gene) )
print ("\n\n",pdbs_with_appris(gene))
one_msa(gene)

print (chimerax(gene))

open "C:/Users/daroc/Documents/Work/Structures/Chimerax/CD44.cxc" 
(['CD44-206'], ['CD44-201', 'CD44-202', 'CD44-203', 'CD44-204', 'CD44-205', 'CD44-207', 'CD44-208', 'CD44-209', 'CD44-210', 'CD44-211', 'CD44-212', 'CD44-213', 'CD44-214', 'CD44-215', 'CD44-216', 'CD44-217', 'CD44-218', 'CD44-219', 'CD44-220', 'CD44-221', 'CD44-222', 'CD44-223', 'CD44-224', 'CD44-225', 'CD44-226', 'CD44-227', 'CD44-228', 'CD44-229', 'CD44-230', 'CD44-231', 'CD44-232', 'CD44-233', 'CD44-234', 'CD44-235', 'CD44-236', 'CD44-237', 'CD44-238'])
Matchces for gene
         qseqid qlen qstart qend    sseqid slen sstart send    evalue bitscore  \
351     1poz_A  159    115  155  CD44-219  243      2   42  7.56e-23     92.0   
4552    2i83_A  160    116  156  CD44-219  243      2   42  6.66e-23     92.0   
256   4pz3_A,B  153    110  153  CD44-231  268      1   44  2.94e-25     98.6   
257   4pz4_A,B  154    110  154  CD44-231  268      1   45  8.37e-26      100   

     length nident gaps gapopen mismatch   pide

In [ ]:
gene = 'CASP1'

print (f"""open "C:/Users/daroc/Documents/Work/Structures/Chimerax/{gene}.cxc" """)
print (ref_nonref(gene))

print ("Matchces for gene\n", matches_for_gene(gene) )
print ("\n\n",pdbs_with_appris(gene))
one_msa(gene)

print (chimerax(gene))


In [ ]:
gene = 'CASP1'

print (f"""open "C:/Users/daroc/Documents/Work/Structures/Chimerax/{gene}.cxc" """)
print (ref_nonref(gene))

print ("Matchces for gene\n", matches_for_gene(gene) )
print ("\n\n",pdbs_with_appris(gene))
one_msa(gene)

print (chimerax(gene))


In [ ]:
print (matches_for_gene(gene, blast_df) )

In [ ]:
# Group by genes
## Note IGHG1 is IGG heavy chain
npp_g2=non_appris_pdbs.groupby('gene', as_index = False) ['qseqid'].nunique()
npp_g2 = npp_g2.rename(columns={'qseqid':'count'}).sort_values(by=['count'], ascending =False)
npp_g2

In [ ]:
npp_g=non_appris_pdbs.groupby('gene', as_index = False) ['sseqid'].nunique()
npp_g = npp_g.rename(columns={'sseqid':'count'}).sort_values(by=['count'], ascending =False)
npp_g

## Export cell output to a file 

Cell 1  
<code>
    %%capture cap --no-stderr
    print(q)
</code>

Cell 2  
<code>
with open('output.txt', 'w') as f:    
    f.write(cap.stdout)
    </code>

### Function to reduce the data to top matches of each PDB
Function is called top_five, the the second option is the numner of hits to include

In [ ]:
def top_five(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            keep.append(True)
        else:
            count += 1
            if count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
#top_five_df = top_five(copy_df)
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

In [ ]:
def top_five_bitscore(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] * 0.95
            keep.append(True)
        else:
            count += 1
            if count<=max and row['bitscore'] > bitscore_threshold:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])
#bitscore_df = top_five_bitscore(blast_df)
#bitscore_df
#top_five_df.to_excel('top5_pdb_vs_gencode.xlsx')

## This will find all isoforms that 
def top_five_bitscore_exact(df, max=5 ):
    keep= []
    last = ""
    for i,row in df.iterrows():
        if row['qseqid'] != last:
            count=1
            last = row['qseqid']
            bitscore_threshold  = row['bitscore'] 
            keep.append(True)
        else:
            count += 1
            if row['bitscore'] == bitscore_threshold and count<=max:
                keep.append(True)
            else:
                keep.append(False)
    return (df.loc[keep])

In [ ]:
top_100_matched = top_five_bitscore_exact(blast_df, 100)
top_100_matched

In [ ]:
top_100_list = []
for i,row in top_100_matched.iterrows():
    top_100_list.append(row.tolist())
print (top_100_list[0])

In [ ]:
print (top_100_list[0])

In [ ]:
top_100_matched.to_csv('Data/top_100_matched.tsv', sep='\t')

In [ ]:
pdb_to_gencode[0]
#pdb_grouped_by_qseqid

In [ ]:
pdb_to_gencode = []
with open('Data/blast_full_PDB_vs_gencode.tsv', 'r') as pdb_file:
    pdb_reader = csv.reader(pdb_file, delimiter='\t')
    next(pdb_reader, None)
    for row in pdb_reader:
        row[5] = row[5].split('|')[0]
        pdb_to_gencode.append(row)
    print (len (pdb_to_gencode))
print (pdb_to_gencode[0:2])

In [ ]:
one_msa('CCR2')

In [ ]:
show_msa(to_inspect, fresh =0, last='EPHA7')

In [ ]:
show_msa(['HARS1'])

In [ ]:
functional=pd.read_csv('Data/APPRIS_FunctionalTableLong.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=0, last='ITGA7')


In [ ]:
functional=pd.read_csv('Data/Tandem_exon_derived_substitution_HES_APPRIS_functional.tsv',sep ='\t')
functional_genes = functional['HGNC'].tolist()
functional_genes
show_msa(functional_genes,fresh=1)


# Below here extra   
mirage, exonerate


In [ ]:
blast_df.groupby(['qseqid'])['sseqid'].aggregate(lambda x: list(x)).reset_index()




## Identify genes with more than one isoform. 

In [ ]:
genes_dict={}
def multiple_isos(df):
    genes=defaultdict(set)
    # Build a dictionary where the values are a set of isoforms for each gene
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            iso=parts[-1]
            #print (id, gene, iso)
            genes[gene].add(iso)
        except:
            pass
    # Create a list (check) of genes that have more than 1 isoform represented.
    check = []
    for g,g_set in genes.items():
        if len(g_set) > 1:
            check.append(g)
            genes_dict[g]=g_set
    print (f'check is {len(check)} long')

    # Create list of df indicies of all genes that have more than one df 
    found=[]
    for i,row in df.iterrows():
        #print (row['sseqid'].split('-')[0])
        try:
            if row['sseqid'].split('-')[0] in check:
                found.append(i)
        except:
            pass
    #print (f'found is {found}')
    found_df = df.loc[found]
    found_df.sort_values(by=['sseqid'], inplace=True)
    return found_df

found_df = multiple_isos(top_100_matched)
found_df

### Find genes that have a structures that match ref and non-ref isoforms 
Note: genes_dict will only contain genes that match more than one isoform.

In [ ]:
to_inspect=[]
#for k in list(genes_dict.keys())[0:10]:
for k in genes_dict:
    try:
        r,nr=ref_nonref(k)
        r_iso = r.split('-')[1]
        if r_iso in genes_dict[k]:
            to_inspect.append(k)
    except Exception as e:
        pass

    

In [ ]:
columns =  ['qseqid','qlen', 'qstart', 'qend', 'sseqid', 'slen', 'sstart', 'send', 'evalue', 'bitscore', 'length', 'nident', 'gaps','gapopen', 'mismatch', 'pident']
filtered = pd.read_csv('Data/filtered_pdb_gencode_mapping.tsv', sep='\t', index_col= False, header = None, names = columns)
filtered

### Write out 

In [ ]:
# Write out the filtered mapping to a TSV file
with open('Data/filtered_pdb_gencode_mapping.tsv', 'w') as output_file:
    output_writer = csv.writer(output_file, delimiter='\t')
    for row in filtered_pdb_to_gencode:
        output_writer.writerow(row)

In [ ]:
gene_dict= defaultdict(set)
def seq_loop(df, dir ='Data/Seq_groups/' ):
    gene_list = []
    for id in df['sseqid']:
        try:
            id = id.split('|')[0]
            parts = id.split('-')
            gene='-'.join(parts[0:-1])
            if gene not in gene_list:
                gene_list.append(gene)
                gene_dict[gene].add(id)
                file = f'Data/Seq_groups/{gene}.fa'
                if os.path.isfile(file):
                    continue
                s = get_gene_seqs(gene)
                with open (f'{dir}/{gene}.fa', 'w') as out:
                    out.write(s)
        except:
            pass
    return (gene_list)
gene_list=seq_loop(found_df, dir)
#filtered_genes= pd.DataFrame(done)  
    

In [ ]:
from tqdm.notebook import trange, tqdm
keep = []
last = ""
for i, row in tqdm(blast_df.iterrows()):
    if row['qseqid'] != last:
        keep.append(True)
        last = row['qseqid']
    else:
        keep.append(False)
one_each = blast_df.loc[ keep]

In [ ]:
def range_mm 
